In [ ]:
import cv2
import dlib
import numpy as np
import tkinter as tk
from tkinter import filedialog, messagebox
from keras.models import load_model
import pandas as pd
from datetime import datetime

In [ ]:
# Load pre-trained models
drowsiness_model = load_model("drowsiness_model.h5")  # Train or download a pre-trained drowsiness detection model
age_net = cv2.dnn.readNetFromCaffe("/kaggle/input/drowsiness-detection/deploy_age.prototxt", "/kaggle/input/drowsiness-detection/age_net.caffemodel")  # Age detection model


In [ ]:
# Age categories
AGE_BUCKETS = ["(0-2)", "(4-6)", "(8-12)", "(15-20)", "(25-32)", "(38-43)", "(48-53)", "(60-100)"]


In [ ]:
# Dlib face detector
face_detector = dlib.get_frontal_face_detector()

In [ ]:
# Helper function: Predict age
def predict_age(face_image):
    blob = cv2.dnn.blobFromImage(face_image, 1.0, (227, 227), (78.4263377603, 87.7689143744, 114.895847746), swapRB=False)
    age_net.setInput(blob)
    age_preds = age_net.forward()
    age = AGE_BUCKETS[age_preds[0].argmax()]
    return age


In [ ]:
# Helper function: Process each frame
def process_frame(frame):
    gray = cv2.cvtColor(frame, cv2.COLOR_BGR2GRAY)
    faces = face_detector(gray)
    
    sleeping_count = 0
    results = []

    for face in faces:
        x, y, w, h = face.left(), face.top(), face.width(), face.height()
        face_image = frame[y:y + h, x:x + w]
        resized_face = cv2.resize(face_image, (64, 64))
        resized_face = resized_face / 255.0
        resized_face = np.expand_dims(resized_face, axis=0)

In [ ]:
 # Predict drowsiness
        drowsiness_pred = drowsiness_model.predict(resized_face)
        asleep = drowsiness_pred[0][0] > 0.5  # Threshold: 0.5 for asleep

        # Predict age
        age = predict_age(face_image)

        if asleep:
            sleeping_count += 1
            color = (0, 0, 255)  # Red for asleep
            label = f"Asleep, Age: {age}"
        else:
            color = (0, 255, 0)  # Green for awake
            label = f"Awake, Age: {age}"

In [ ]:
 # Draw rectangle and label
        cv2.rectangle(frame, (x, y), (x + w, y + h), color, 2)
        cv2.putText(frame, label, (x, y - 10), cv2.FONT_HERSHEY_SIMPLEX, 0.6, color, 2)

        results.append({"asleep": asleep, "age": age})

    return frame, results, sleeping_count
